# Дополнительное задание: SQL

# Задача
Изучить базу с данными, которые помогут сформулировать ценностное предложение для нового продукта.

# Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

# Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

# Изучаю таблицы

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
query = '''
SELECT *
FROM books
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
query = '''
SELECT *
FROM authors
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
query = '''
SELECT *
FROM publishers
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
query = '''
SELECT *
FROM ratings
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
query = '''
SELECT *
FROM reviews
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Сколько книг вышло после 1 января 2000 года

In [7]:
query = '''
SELECT 
    COUNT(book_id) AS books_amount
FROM 
    books
WHERE
    CAST(publication_date  AS date) > '2000-01-01'
'''   
pd.io.sql.read_sql(query, con = engine)

,books_amount
0,819


819 книг из нашей выборки вышли после 01.01.2020

## Количество обзоров и средняя оценка каждой книги

In [109]:
query = '''
SELECT    
    total.book_id,
    total.title, 
    total.count,
    AVG(ratings.rating)
FROM
    (SELECT 
        books.book_id,
        books.title, 
        COUNT(reviews.review_id) as count
    FROM 
        books 
        JOIN reviews on books.book_id = reviews.book_id
    GROUP BY
        books.book_id
    ORDER BY
        count desc) as total
    JOIN ratings on total.book_id = ratings.book_id
GROUP BY
    total.book_id,
    total.title,
    total.count
ORDER BY
    total.count desc
    
'''   
pd.io.sql.read_sql(query, con = engine)

,book_id,title,count,avg
0,948,Twilight (Twilight #1),7,3.662500
1,734,The Glass Castle,6,4.206897
2,733,The Giver (The Giver #1),6,3.750000
3,696,The Da Vinci Code (Robert Langdon #2),6,3.830508
4,656,The Book Thief,6,4.264151
...,...,...,...,...
989,823,The Philosophy of Andy Warhol (From A to B and...,1,4.500000
990,156,Color: A Natural History of the Palette,1,4.333333
991,92,Babyville,1,3.500000
992,855,The Road Not Taken and Other Poems,1,3.500000


Получили таблицу с 2 новыми калонками: среднее оценка книги и число написанных на неё обзоров. 

Больше всего обзоров получили Сумерки -- целых 7 штук, хотя рейтинг оставляет желать лучшего -- всего 3,66. 

## Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [9]:
query = '''
SELECT
    publishers.publisher,
    COUNT(books.book_id) as count
FROM 
    publishers 
    JOIN books on publishers.publisher_id = books.publisher_id
WHERE
    books.num_pages > 50
GROUP BY
    publishers.publisher_id
ORDER BY
    count desc
LIMIT 1

'''   
pd.io.sql.read_sql(query, con = engine)

,publisher,count
0,Penguin Books,42


Самые продуктивные -- английское издательство Penguin Books. 

## Автор с самой высокой средней оценкой книг c 50 и более оценками

In [28]:
query = '''
SELECT
    top_autors.author,
    AVG(top_autors.avg_rate)
FROM    
    (SELECT
        authors.author,
        top_book.avg_rate
    FROM
        (SELECT
            rate_books.book_id,
            AVG(ratings.rating) as avg_rate
        FROM
            (SELECT    
                books.book_id,
                COUNT(ratings.rating) 
            FROM
                books
                JOIN ratings on books.book_id = ratings.book_id
            GROUP BY
                books.book_id
            HAVING
                COUNT(ratings.rating) > 50) as rate_books
            JOIN ratings on ratings.book_id = rate_books.book_id
        GROUP BY
            rate_books.book_id) as top_book
        JOIN books on books.book_id = top_book.book_id
        JOIN authors on authors.author_id = books.author_id) as top_autors
GROUP BY
    top_autors.author
ORDER BY
    avg desc
LIMIT 1

'''   
pd.io.sql.read_sql(query, con = engine)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


Самая высокая оценка у Роулинг. Сложно найти кого-то, кому не нравится Гарри Поттер (мне например ;)) 

## Cреднее количество обзоров от пользователей c 50 и более оценками

In [11]:
query = '''
SELECT
    AVG(average.count)
FROM
    (SELECT    
        total.username,
        COUNT(total.username)
    FROM   
        (SELECT 
            username,
            COUNT(rating_id) as count
        FROM 
            ratings
        GROUP BY
            username) as total
        JOIN reviews on reviews.username = total.username
    WHERE 
        total.count > 50
    GROUP BY
        total.username) as average
    
'''   
pd.io.sql.read_sql(query, con = engine)

,avg
0,24.333333


В среднем активные пользователи (оставившие более 50 оценок) написали 24 обзора

# Вывод 

- Посчитал, сколько книг вышло после 1 января 2000 года;

- Для каждой книги посчитал количество обзоров и среднюю оценку;

- Определил издательство, которое выпустило наибольшее число книг толще 50 страниц;

- Определил автора с самой высокой средней оценкой книг;

- Посчитал среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

Каждый расчёт сделал одним SQL-запросом. По каждому пункту рассчётов сделал вывод.